# A Network Graph for VCs and Angels

This notebook looks to give insight into some basics of <b>social network analysis</b>. <i>Social network analysis examines the structure of relationships between social entities. These entities are often persons, but may also be groups, organizations, nation states, web sites, or scholarly publications.</i>

<b>The notebook will cover some simple social network analysis regarding the start up scene</b>. If you follow tech, you know some schools look like incubators at times; Stanford, MIT, etc. Is this due to the fact that VCs and Angels are alumni of these schools, and either starups or VCs are leveraging their network, which is why so many startups are connected to these schools. Possibly. This question is at the heart of this notebook's analysis, and this notebook will not answer this question but rather start showing some tools that can help if one decides to try.

The notebook will mostly use tools from: SFrame (graphlab-create), networkx, and numpy.
One could use Panda instead of SFrame. 
All data is publically available and will be from Techcrunch. Also, this notebook will show only subsets of the data at time for some of the analysis.

## Importing the data

The data was downloaded from techcrunch, and is in json format. The overall idea is to see if VCs or angels are connected to the startups they gave money to by schools they attended. Simply, are the VCs and angels alumni of the same schools as the startup founders. There are a four json files. One for founders and their education, one for vc firms and their education, one for people (angels) and their education, and one showing investments between groups. 

Below, the data will be imported into SFrames, cleaned, and joined to make a table holding the connections between investors and investees. Then we will create a new table that only has 

In [1]:
import graphlab as gl
import numpy as np

#read in education of firms from json
sf=gl.SFrame.read_json('companies_person_educ.json', orient='records')

#remove useless columns
sf=sf.remove_columns(['company_name', 'degree_type','graduated_at', 'person_name', 'title'])
sf['institution']=sf['institution'].apply(lambda x: x.lower())

#fix university with ids
un=gl.SFrame.read_csv('degreematching.csv')
#drop inst useless columns
un=un.remove_columns(['line', 'inst_name'])
#make lower for matching purposes
un['institution']=un['institution'].apply(lambda x: x.lower())

#match school names
sf=sf.join(un, on='institution', how='left')

#drop useless col
sf=sf.remove_column('institution')

#unstack on institution
sf=sf.unstack('inst_id', new_column_name='schools_id')

# read in ed of vc from json
vc=gl.SFrame.read_json('finorgs_person_educ.json', orient='records')

#remove useless columns
vc=vc.remove_columns(['finorgs_name', 'degree_type','graduated_at', 'person_name', 'title'])

#
vc['institution']=vc['institution'].apply(lambda x: x.lower())

#match school names
vc=vc.join(un, on='institution', how='left')

#
vc=vc.remove_column('institution')


#unstack on institution
vc=vc.unstack('inst_id', new_column_name='schools_id')

#read in connection from vc to firms
invest=gl.SFrame.read_json('investments.json', orient='records')

#remove useless columns
invest=invest.remove_columns(['created_at', 'id','funding_round_id', 'updated_at'])

#remove duplicates
invest=invest.unique()

#rename invest titles
invest=invest.rename({'funded_object_id':'company_id', 'investor_object_id': 'finorgs_id'})

#clean angel info
angels=gl.SFrame.read_json('persons_educ.json', orient='records')
angels.remove_columns(['created_at','degree_type','graduated_at','id','subject','updated_at'])
angels['institution']=angels['institution'].apply(lambda x: x.lower())
angels=angels.join(un, on='institution', how='left')
angels=angels.dropna()
angels['inst_id']=angels['inst_id'].apply(lambda x: None if x=='' else x)
angels=angels.dropna()

angels.remove_column('institution')
angels=angels.unstack('inst_id', new_column_name='schools_id')
angels['schools_id']=angels['schools_id'].apply(lambda x: np.unique(x))

#filter p,c,f
invest['test']=invest['finorgs_id'].apply(lambda x: x[0])
t=invest.filter_by('c','test')
t.remove_column('test')
tp=invest.filter_by('p','test')
tp.remove_column('test')
tf=invest.filter_by('f','test')
tf.remove_column('test')

#joins
sf1=sf.sort('company_id')
sf1=sf1.rename({'company_id':'finorgs_id'})
angels1=angels.sort('object_id')
angels1=angels1.rename({'object_id':'finorgs_id'})

tp=tp.join(angels1,on='finorgs_id', how='left')
tp['person_id']=tp['finorgs_id']
t=t.join(sf1,on='finorgs_id', how='left')
tf=tf.join(vc,on='finorgs_id', how='left')

rightside=t.append(tp)
rightside=rightside.append(tf)
rightside=rightside.rename({'schools_id': 'schools_of_vcs','person_id':'vc_ids'})

total=rightside.join(sf, on='company_id', how='left')


#rename sf columns
total=total.rename({'schools_id': 'schools_of_founders', 'person_id':'founder_ids'})

#strip duplicates
total['schools_of_vcs']=total['schools_of_vcs'].apply(lambda x: np.unique(x))
total['schools_of_founders']=total['schools_of_founders'].apply(lambda x: np.unique(x))

total=total.dropna()

#clean func
def clean_arr( arrlist):
    arr=[]
    x=list(set(arrlist))
    for xx in x:
	if len(xx)>1:
	    arr.append(xx)
    return arr

#use clean function
total['schools_of_founders']=total['schools_of_founders'].apply(lambda x: clean_arr(x))

total['schools_of_vcs']=total['schools_of_vcs'].apply(lambda x: clean_arr(x))

def intersect(a, b):
    return list(set(a) & set(b))

#drop None
total=total.dropna()

#make connections
total['connected_by_schools']=total.apply(lambda x: intersect( x['schools_of_founders'], x['schools_of_vcs']))

#count contections between ind
total['number_of_same_schools']=total['connected_by_schools'].apply(lambda x: len(x))

[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1466636207.log


This non-commercial license of GraphLab Create is assigned to Skorupry@uw.edu and will expire on December 17, 2016. For commercial licensing options, visit https://dato.com/buy/.


Finished parsing file /home/ryan/Portfolio/KnowGraph/companies_person_educ.json

Parsing completed. Parsed 1 lines in 2.51325 secs.

Finished parsing file /home/ryan/Portfolio/KnowGraph/degreematching.csv

...

Parsing completed. Parsed 1 lines in 0.375487 secs.

Finished parsing file /home/ryan/Portfolio/KnowGraph/investments.json

Parsing completed. Parsed 1 lines in 0.468361 secs.

Finished parsing file /home/ryan/Portfolio/KnowGraph/persons_educ.json

Parsing completed. Parsed 1 lines in 0.779809 secs.

In [2]:
#show the table with the results
total.head()

company_id,finorgs_id,vc_ids,schools_of_vcs,founder_ids,schools_of_founders,connected_by_schools,number_of_same_schools
c:29150,c:10022,p:48956,[15231],p:69068,[13614],[],0
c:29150,c:10022,p:48956,[15231],p:117930,[14557],[],0
c:29150,c:10022,p:177084,[15231],p:69068,[13614],[],0
c:29150,c:10022,p:177084,[15231],p:117930,[14557],[],0
c:29150,c:10022,p:166859,[14147],p:69068,[13614],[],0
c:29150,c:10022,p:166859,[14147],p:117930,[14557],[],0
c:17957,c:10071,p:17001,[],p:256683,[],[],0
c:17957,c:10071,p:17001,[],p:256682,[15865],[],0
c:17957,c:10071,p:17001,[],p:256681,[],[],0
c:17957,c:10071,p:17001,[],p:256684,[],[],0


## Understanding the data

Now that the data is in a form that gives us the connections on education, let's make sure we understand the table. The 'company_id' column gives a reference id associated with the startup. The 'finorgs_id' gives the reference id associated with the firm that gave the money. The 'vc_ids' gives the reference id for the person. The 'schools_of_vcs' column gives a vector of the school ids that the vc went to. The 'founder_ids' column gives the reference id of the person from the startup. The 'schools_of_founders' column gives a vector of the school ids that the founder went to. The 'connected_by_schools' column gives the intersection of the two school columns. Lastly, the 'number_of_same_schools' cloumn gives the length of the previous coulmn; in other words the number of school the two share. For example, if a VC went to Cornel for undergrad and Stanford for his MBA and a founder went to Stanford for his undergrad, NYU for his masters, and Cornel for his JD then the two would share two schools. 

Now, let's create a subset of the data and form a network graph using networkx.

In [3]:
#find the largest vc firms by amount they appear in the table
total['finorgs_id'].sketch_summary()


+------------------+---------+----------+
|       item       |  value  | is exact |
+------------------+---------+----------+
|      Length      | 6694193 |   Yes    |
| # Missing Values |    0    |   Yes    |
| # unique values  |   4803  |    No    |
+------------------+---------+----------+

Most frequent items:
+-------+--------+--------+--------+--------+--------+--------+--------+--------+
| value |  c:59  |  f:36  |  f:41  | c:1242 |  c:28  |  f:17  | f:316  |  f:4   |
+-------+--------+--------+--------+--------+--------+--------+--------+--------+
| count | 490287 | 361218 | 312573 | 294810 | 260550 | 259080 | 212220 | 204332 |
+-------+--------+--------+--------+--------+--------+--------+--------+--------+
+--------+--------+
| f:367  | f:563  |
+--------+--------+
| 163576 | 100080 |
+--------+--------+


## Choosing a subset

To create a graph, let's just look at one vc firm. 'f:563' has 100,080 observations, and seems reasonable to filter on that.

In [4]:
subset=total.filter_by('f:563', 'finorgs_id')

In [5]:
#make sure there are some connections in the subset
connectedsubset=subset.filter_by([1,2,3,4,5,6,7,8,9], 'number_of_same_schools')

connectedsubset

company_id,finorgs_id,vc_ids,schools_of_vcs,founder_ids,schools_of_founders,connected_by_schools,number_of_same_schools
c:1812,f:563,p:4047,"[14317, 16503]",p:28195,"[14317, 15808]",[14317],1
c:1812,f:563,p:4047,"[14317, 16503]",p:151424,"[14317, 16256]",[14317],1
c:1812,f:563,p:4047,"[14317, 16503]",p:799,"[14317, 14814]",[14317],1
c:1812,f:563,p:4047,"[14317, 16503]",p:186675,[14317],[14317],1
c:1812,f:563,p:4047,"[14317, 16503]",p:11713,"[16503, 15808]",[16503],1
c:1812,f:563,p:4047,"[14317, 16503]",p:2603,[14317],[14317],1
c:1812,f:563,p:245517,[16116],p:158061,[16116],[16116],1
c:1812,f:563,p:245517,[16116],p:123213,[16116],[16116],1
c:1812,f:563,p:242516,[14317],p:28195,"[14317, 15808]",[14317],1
c:1812,f:563,p:242516,[14317],p:151424,"[14317, 16256]",[14317],1


## EDA

Therefore, out of 100,800 observations there exist conections 5,547 times.

In [6]:
#find out how many vcs are in the firm
len(list(subset['vc_ids'].unique()))

278

In [7]:
#find out how many unique startups this firm invests in
len(list(subset['company_id'].unique()))

25

In [8]:
#find out how many unique employees/founders there are
len(list(subset['founder_ids'].unique()))

355

In [9]:
#find out how many unique schools there are
schools=list(list(subset['schools_of_vcs'])+list(subset['schools_of_founders']))

arr=[]
for s in schools:
    for ss in s:
        arr.append(ss)
        
schools=list(set(arr))

len(schools)

177

In [10]:
#match ids with school names
#fix university with ids
uni=gl.SFrame.read_csv('degreematching.csv', column_type_hints=[str,int,int,str])
#drop inst useless columns
uni=uni.remove_columns(['institution', 'line'])
uni=uni.dropna()
uni['inst_name']=uni['inst_name'].apply(lambda x: str(x).lower())
uni=uni.unique()

#get number alumni at each school for vcs
vc=subset[['vc_ids', 'schools_of_vcs']]
vc=vc.unique()
vc=vc.stack('schools_of_vcs',new_column_name='inst_id')
vc=vc.remove_column('vc_ids')

vc['inst_id']=vc['inst_id'].apply(lambda x: int(x))
vc_schools=vc.join(uni, on='inst_id', how='left')


vc_schools=vc_schools.unstack('inst_id',new_column_name='inst_id')

vc_schools['num_vc_alumni']=vc_schools['inst_id'].apply(lambda x: len(x))
vc_schools.sort('num_vc_alumni',ascending=False)

Finished parsing file /home/ryan/Portfolio/KnowGraph/degreematching.csv

Parsing completed. Parsed 17710 lines in 0.016172 secs.

inst_name,inst_id,num_vc_alumni
harvard university **,"[14317, 14317, 14317,14317, 14317, 14317, ...",52
stanford university,"[15808, 15808, 15808,15808, 15808, 15808, ...",42
university ofpennsylvania ...,"[16162, 16162, 16162,16162, 16162, 16162, ...",14
columbia university newyork ...,"[13837, 13837, 13837,13837, 13837, 13837, ...",13
yale university,"[16503, 16503, 16503,16503, 16503, 16503, ...",12
princeton university,"[15403, 15403, 15403,15403, 15403, 15403, ...",10
cornell university,"[13887, 13887, 13887,13887, 13887, 13887, ...",9
new york university,"[15098, 15098, 15098,15098, 15098, 15098, ...",9
massachusetts instituteof technology ...,"[14814, 14814, 14814,14814, 14814, 14814, ...",8
dartmouth college,"[13937, 13937, 13937,13937, 13937, 13937, ...",8


The above table informs us that a majority of the vcs at this firm are Harvard and Stanford alumn. Let's see where the majority of founders are alum from.

In [11]:
#get number alumni at each school for founders
fo=subset[['founder_ids', 'schools_of_founders']]
fo=fo.unique()
fo=fo.stack('schools_of_founders',new_column_name='inst_id')
fo=fo.remove_column('founder_ids')

fo['inst_id']=fo['inst_id'].apply(lambda x: int(x))
fo_schools=fo.join(uni, on='inst_id', how='left')


fo_schools=fo_schools.unstack('inst_id',new_column_name='inst_id')

fo_schools['num_founder_alumni']=fo_schools['inst_id'].apply(lambda x: len(x))
fo_schools.sort('num_founder_alumni',ascending=False)

inst_name,inst_id,num_founder_alumni
stanford university,"[15808, 15808, 15808,15808, 15808, 15808, ...",64
harvard university **,"[14317, 14317, 14317,14317, 14317, 14317, ...",32
university ofpennsylvania ...,"[16162, 16162, 16162,16162, 16162, 16162, ...",14
massachusetts instituteof technology ...,"[14814, 14814, 14814,14814, 14814, 14814, ...",14
new york university,"[15098, 15098, 15098,15098, 15098, 15098, ...",11
university of michigan,"[16116, 16116, 16116,16116, 16116, 16116, ...",11
columbia university newyork ...,"[13837, 13837, 13837,13837, 13837, 13837, ...",10
cornell university,"[13887, 13887, 13887,13887, 13887, 13887, ...",9
university of illinoisurbana champaign * ...,"[16083, 16083, 16083,16083, 16083, 16083, ...",8
georgetown university,"[14216, 14216, 14216,14216, 14216, 14216, ...",8


It appears the founders are also concentrated as alum at mainly Stanford and Harvard.

## Creating the Graph

Above we figured out some basic information about the subset of data we had. Now it is time to start created the graph. The graph will be very large even with this subset of people for that reason, we do the graph work on one vc.



In [12]:
#filter on one vc
sub=connectedsubset.filter_by(['p:57950'],'vc_ids' )

sub

company_id,finorgs_id,vc_ids,schools_of_vcs,founder_ids,schools_of_founders,connected_by_schools,number_of_same_schools
c:1812,f:563,p:57950,"[15403, 15808]",p:243095,[15808],[15808],1
c:1812,f:563,p:57950,"[15403, 15808]",p:28195,"[14317, 15808]",[15808],1
c:1812,f:563,p:57950,"[15403, 15808]",p:7991,"[15098, 15403]",[15403],1
c:1812,f:563,p:57950,"[15403, 15808]",p:93262,[15808],[15808],1
c:1812,f:563,p:57950,"[15403, 15808]",p:16518,"[16232, 15808]",[15808],1
c:1812,f:563,p:57950,"[15403, 15808]",p:570,[15808],[15808],1
c:1812,f:563,p:57950,"[15403, 15808]",p:45734,[15808],[15808],1
c:1812,f:563,p:57950,"[15403, 15808]",p:197898,[15403],[15403],1
c:1812,f:563,p:57950,"[15403, 15808]",p:59991,"[14733, 15808]",[15808],1
c:1812,f:563,p:57950,"[15403, 15808]",p:20927,"[16231, 15808]",[15808],1


In [13]:
#unique companies in sub
len(list(sub['company_id'].unique()))

15

In [14]:
import networkx as nx
import matplotlib.pyplot as plt

G=nx.Graph() #create graph

#add nodes from the list of vcs
G.add_nodes_from(list(sub['vc_ids'].unique()), color='green') 

#number of vcs in subset
len(list(sub['vc_ids'].unique()))

1

In [15]:
plt.clf()
nx.draw(G)
#plt.show()
plt.savefig("vc.png")

### Graph of VC

<img src="vc.png" alt="vc" style="width:600px;height:400px;">
The code above generate one node. This node is are VC.

In [17]:
#create function to create 3-tuple to create edges in networkx
def tup(alist ):
    arr=[]
    while len(alist)>0:
        index=1
        while index<len(alist):
            arr.append((alist[0],alist[index]))
            index+=1
        alist=alist[1::]
    return arr

#color='g',weight=6

In [18]:
#create nodes and edges for the companies in the subset
uncomps=list(sub['company_id'].unique())

for un in uncomps:
    c=sub.filter_by(un,'company_id' ) 
    G.add_nodes_from(list(c['founder_ids'].unique())) 
    G.add_edges_from(tup(list(c['founder_ids'].unique())))

In [19]:
plt.clf()
nx.draw(G)
#plt.show()
plt.savefig("comps.png")

### Graph with companies connected

<img src="comps.png" alt="comp" style="width:600px;height:400px;">

As we can see there are a few large comopanies and and some very small ones. Now let's connect them to the one vc by school connection. Also note, networkx automatically picks the way the graph looks if we do not speicify and clusters the connected together.

In [20]:
#create list to filter for
conby=list(sub['connected_by_schools'].unique())
conby

[['15403'], ['15403', '15808'], ['15808']]

In [21]:
#schools vc is connected by
uni.filter_by(15403,'inst_id')

inst_id,inst_name
15403,princeton university


In [22]:
#schools vc is connected by
uni.filter_by(15808,'inst_id')

inst_id,inst_name
15808,stanford university


In [23]:
#create function to create tuple to connect vc and founders through education
def ed(vc, inst_code):
    arr=[]
    fff=sub.filter_by(inst_code, 'connected_by_schools')
    for ff in list(fff['founder_ids'].unique()):
        arr.append((vc,ff))
    return arr

In [24]:
#code to connect based on either princeton, stanford, both
#princeton
plt.clf()
G.add_edges_from(ed('p:57950',[['15403']]))
nx.draw(G)
plt.savefig("princeton.png")

### Graph of Princeton

<img src="princeton.png" alt="princ" style="width:600px;height:400px;">

The VC is connected to five princeton alum at companies.

In [25]:
#code to connect based on either princeton, stanford, both
#stanford
plt.clf()
G.add_edges_from(ed('p:57950',[['15808']]))
nx.draw(G)
plt.savefig("stanford.png")

### Graph of Stanford

<img src="stanford.png" alt="stan" style="width:600px;height:400px;">

The VC is connected to a lot more people through his Stanford network.

In [30]:
#Code to construct the graph differently (all code here is commented out) 
#plt.clf()
#limits=plt.axis('off') 

#pos = nx.spring_layout(G)
#nx.draw_networkx(G, pos=pos, nodelist=list(sub['vc_ids'].unique()), node_size=1000, node_color='#A0CBE2', font_size=0, width=2)

In [31]:
#plt.show()

# Conclusion

Honestly, there are hundreds of other things to do to take this analysis/exploration further. Then first would be doing EDA on the entire sample and then building the entire graph and and examining it. 

Anyway, this notebook's purpose is just to help get you started. Happy exploring. 